In [149]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

In [150]:
def myseq_with_plotting(pixelMatrix,grayLevels = 256):
    H,W = pixelsMatrix.shape
    size = H*W
    occurrencies = [0.0 for _ in range(grayLevels)]
    pixels_flattened = pixelMatrix.flatten()
    for pixel in pixels_flattened:
        occurrencies[pixel]+=1
    cdf = [sum(occurrencies[:i+1]) for i in range(grayLevels)]
    cdfmin = next((x for x in cdf if x),-1)
    h = [round((cdf[v]-cdfmin)/(size-cdfmin) * (grayLevels-1)) for v in range(grayLevels)]

    for id,pixel in enumerate(pixels_flattened):
        pixels_flattened[id] = h[pixel]

    occurrencies_equalized = [0.0 for _ in range(grayLevels)]
    for pixel in pixels_flattened:
        occurrencies_equalized[pixel]+=1

    reshaped_image = np.reshape(pixels_flattened,(W,H))
    
    plt.bar(range(grayLevels),occurrencies_equalized,label = 'equalized')
    plt.bar(range(grayLevels),occurrencies,label = 'original')
    plt.legend()
    return Image.fromarray(reshaped_image)


In [151]:
def sequential(pixelMatrix,grayLevels = 256) -> np.ndarray:
    H,W = pixelMatrix.shape
    size = H*W
    occurencies = [0.0 for _ in range(grayLevels)]
    pixels_flattened = pixelMatrix.flatten()
    for pixel in pixels_flattened:
        occurencies[pixel]+=1
    cdf = [int(sum(occurencies[:i+1])) for i in range(grayLevels)]
    cdfmin = next((x for x in cdf if x),-1)

    h = [round((cdf[v]-cdfmin)/(size-cdfmin) * (grayLevels-1)) for v in range(grayLevels)]
    for id,pixel in enumerate(pixels_flattened):
        pixels_flattened[id] = h[pixel]
    return np.reshape(pixels_flattened,(W,H))
    


In [152]:
import numba as  nb
import numpy as np
from numba import cuda

@cuda.jit
def countOccurencies(pixelsMatrix,occurencies):
    x,y = cuda.grid(2)
    if x>=pixelsMatrix.shape[0] or y>=pixelsMatrix.shape[1]:
        return
    cuda.atomic.add(occurencies,pixelsMatrix[x][y],1)

@cuda.jit
def calcCDF(occurencies_d,cdf):
    acc = 0
    for i in range(cuda.grid(1)+1):
        acc+=occurencies_d[i]
    cdf[cuda.grid(1)] = acc

@cuda.jit
def calcH(h,cdf_d,cdfmin_d,size_d,grayLevels_d):
    nominator = (cdf_d[cuda.grid(1)]-nb.int32(cdfmin_d))

    denominator = (size_d-cdfmin_d)
    multiplier = (grayLevels_d-1)
    result = round(nominator/denominator*multiplier)

    h[cuda.grid(1)] = result
    # print("XD")

@cuda.jit
def changeOriginalValues(h_d,pixelsMatrix):
    x,y = cuda.grid(2)
    if x>=pixelsMatrix.shape[0] or y>=pixelsMatrix.shape[1]:
        return
    pixelsMatrix[x][y] = nb.int32( h_d[pixelsMatrix[x][y]] )





def parallel(pixelsMatrix,grayLevels = 256):
    H,W = pixelsMatrix.shape
    blockdim = (32, 32)

    griddim = (H // blockdim[0] + 1,W // blockdim[1] + 1)
    occurencies = np.zeros(grayLevels,np.int32)
    pixelsMatrix = pixelsMatrix.astype(np.int32)
    pixelsMatrix_d = cuda.to_device(pixelsMatrix)
    occurencies_d = cuda.to_device(occurencies)
    countOccurencies[griddim,blockdim](pixelsMatrix_d,occurencies_d)
    cdf = np.zeros(grayLevels,np.int32)
    cdf_d = cuda.to_device(cdf)
    threadsperblock = 32
    blockspergrid = (grayLevels + (threadsperblock - 1)) // threadsperblock
    calcCDF[threadsperblock,blockspergrid](occurencies_d,cdf_d)

    del occurencies_d

    cdfmin = next((x for x in cdf_d if x),-1)

    h = np.zeros(grayLevels,np.int32)
    h_d = cuda.to_device(h)
    

    calcH[threadsperblock,blockspergrid](h_d,cdf_d,nb.int32(cdfmin),nb.int32(H*W),nb.int32(grayLevels))
    del cdfmin
    del cdf_d
    changeOriginalValues[griddim,blockdim](h_d,pixelsMatrix_d)
    
    del h_d


    return pixelsMatrix_d




In [153]:
hugeIMG = Image.open('huge_image.png').convert('L')
pixelsMatrix = np.array(hugeIMG)

In [154]:
%%time 
xd = parallel(pixelsMatrix)

CPU times: user 346 ms, sys: 0 ns, total: 346 ms
Wall time: 346 ms


In [155]:
%%time
sequ = sequential(pixelsMatrix)

CPU times: user 600 ms, sys: 0 ns, total: 600 ms
Wall time: 599 ms


In [156]:
einstein = Image.open('einstein.png').convert('L')
pixelsMatrix = np.array(einstein)

In [157]:
%%time
sequ = parallel(pixelsMatrix)

CPU times: user 3.23 ms, sys: 0 ns, total: 3.23 ms
Wall time: 2.71 ms


In [158]:
%%time
sequ = sequential(pixelsMatrix)

CPU times: user 13.7 ms, sys: 0 ns, total: 13.7 ms
Wall time: 13.5 ms


In [174]:
stars = Image.open('stars.png').convert('L')
pixelsMatrix = np.array(stars)

In [179]:
%%time
for _ in range(100):
    sequ = parallel(pixelsMatrix)
print(np.array(sequ))

[[64 83 43 ... 64 64 83]
 [64 64 43 ... 64 64 64]
 [21 43 83 ... 83 64 64]
 ...
 [21 21 21 ... 43 43 64]
 [ 5 21 21 ... 43 83 43]
 [21 43 21 ... 64 64 83]]
CPU times: user 732 ms, sys: 47.4 ms, total: 780 ms
Wall time: 784 ms


In [180]:
%%time
for _ in range(100):
    sequ = sequential(pixelsMatrix)
print(sequ)

[[64 83 43 ... 64 64 83]
 [64 64 43 ... 64 64 64]
 [21 43 83 ... 83 64 64]
 ...
 [21 21 21 ... 43 43 64]
 [ 5 21 21 ... 43 83 43]
 [21 43 21 ... 64 64 83]]
CPU times: user 14.5 s, sys: 0 ns, total: 14.5 s
Wall time: 14.5 s
